In [18]:
import torch
import cv2
from torchvision import transforms
from facenet_pytorch import MTCNN

# Загрузка предварительно обученной модели MTCNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

# Загрузка видео
video_path = r"C:\Users\User\Desktop\test_video.mp4"  # Замените на путь к вашему видео
cap = cv2.VideoCapture(video_path)

# Получение fps и размера кадра из исходного видео
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_blurred.mp4", fourcc, fps, (frame_width, frame_height))

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Достигнут конец видео.")
            break

        # Преобразование кадра в тензор и обнаружение лиц
        img = transforms.ToPILImage()(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        boxes, _ = mtcnn.detect(img)

        # Применение блюра к обнаруженным лицам
        if boxes is not None:
            for box in boxes:
                x, y, w, h = int(box[0]), int(box[1]), int(box[2] - box[0]), int(box[3] - box[1])
                face_roi = frame[y:y+h, x:x+w]

                # Проверка, что face_roi не пусто
                if face_roi is not None and not face_roi.size == 0:
                    blurred_face = cv2.GaussianBlur(face_roi, (99, 99), 30)  # Параметры блюра можно настроить
                    frame[y:y+h, x:x+w] = blurred_face

        # Запись кадра в файл
        out.write(frame)

        cv2.imshow('Blurred Faces', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    print(f"Произошла ошибка: {e}")

cap.release()
out.release()
cv2.destroyAllWindows()

Достигнут конец видео.


In [19]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
